In [16]:
!pip install --upgrade openai 
!pip install python-dotenv
!pip install pydantic

This Import Dependencies

In [3]:
import openai
import os
import json
import logging
import pandas as pd
from dotenv import load_dotenv
from resources.OpenAIAzureEngine import OpenAIAzureEngine
from resources.OpenAIService import OpenAIService
from resources.CambridgeAssessmentCriteria import CambridgeAssessmentCriteria
from resources.WritingCriteriaScore import WritingCriteriaScore 
from resources.WritingResult import WritingResult

In [4]:
load_dotenv()

True

In [5]:
azure_api_key = os.getenv("OPEN_AI_API_KEY")

In [6]:
engine_text=OpenAIAzureEngine(api_version="2024-08-01-preview", deployment="gpt-4o-2", api_key=azure_api_key)

service = OpenAIService(engine_text=engine_text)


In [7]:
def convert_openai_output_to_df(openai_output):
    # Extracting keys and values from the input dictionary
    criteria = list(openai_output.keys())
    scores = [openai_output[crit]['score'] for crit in criteria]
    feedback = [openai_output[crit]['feedback'] for crit in criteria]
    
    # Creating a DataFrame
    data = {
        "Criteria": criteria,
        "Score": scores,
        "Feedback": feedback
    }
    
    # Returning the DataFrame
    return pd.DataFrame(data)

In [8]:
def get_assessment_criteria() -> str:
    criteria  = {
            CambridgeAssessmentCriteria.Content.value: "Does the answer fulfill the task requirements? Is the response relevant and complete? Ensure the TaskAnswer is not just a repetition of the TaskAssessment. If it is, deduct points and provide feedback.",
            CambridgeAssessmentCriteria.CommunicativeAchievement.value: "Is the language appropriate for the task? Does the answer address the context? Check for originality and appropriate extensions of the TaskAssessment.",
            CambridgeAssessmentCriteria.Organization.value: " Is the response coherent and well-organized? Are ideas logically connected? If the answer repeats the TaskAssessment, penalize for lack of coherence.",
            CambridgeAssessmentCriteria.Language.value: " Is the grammar accurate and vocabulary appropriate for the level? Ensure that new vocabulary or grammar structures are used in the answer compared to the task.",
    }
    
    if isinstance(criteria, dict):
        return '\n\t\t'.join(f"--{key}: \"{value}\"" for key, value in criteria.items())
    return ""

In [9]:
question=""" Rubric:  
Articles wanted!

WHAT MAKES YOU LAUGH?

Write an article telling us what you find funny and who you enjoy laughing with.

Do you think it’s good to laugh a lot? Why?

The best articles answering these questions will be published next month.

Write your article.
"""

In [10]:
student_answer="""
What makes me laugh? Pretty much everything. I can start laughing simply from watching my cat trying to catch a laser dot on the wall, or a fly. And I laugh even more with my friends and family. It is practically the only way we are spending time with ourselves. We are making jokes, laughing at something or telling funny stories about our trips, days at school or our pets.

I think it is important to laugh a lot, because it makes everyday stress easier to handle or it simply wears off! And it makes you look happy, optimistic and ready for a challenge.
"""

In [11]:

def get_user_prompt(question: str = '', answer: str = '') -> str:
    return f"""
The task requires candidates to write an article of approximately 100 words in response to an announcement or prompt, typically from a magazine or website. The input outlines the topic of the article and usually includes a set of questions that candidates must address in their writing.

Candidates are expected to write an engaging and informative article that is suitable for the specified audience. The article should be well-structured, with a clear introduction, body, and conclusion, and should directly respond to all the points in the prompt.

The task is assessed using four subscales: Content, Communicative Achievement, Organisation, and Language. Candidates should focus on using a tone appropriate for an article, clear and logical organization, and a range of vocabulary and grammar reflective of the B1 level.

Candidates are expected to write an article in response to the prompt, answering the questions provided and adhering to an approximate word count of 140-190 words.
**Context:**

- The response must be evaluated based on Cambridge's B1 assessment scales. The scales consist of four subscales: **Content**, **Communicative Achievement**, **Organisation**, and **Language**.
- You are expected to provide a **score (0-5)** for each subscale and brief constructive **feedback** (no more than 5 sentences).
- The **task instructions** and a **one-shot example** of evaluations have been provided to guide your assessment.
- Use the **Cambridge Assessment B1 Preliminary handbook** and the **Common European Framework of Reference (CEFR)** to support your evaluation.
- **Identify all specific task points from the task instructions and assess whether the student's response addresses them fully and explicitly.**
- **PAY SPECIAL ATTENTION TO THE CONTENT SUBSCALE, ENSURING THAT ALL TASK POINTS ARE ADDRESSED.**
- **USE A CHECKLIST TO CONFIRM THAT ALL REQUIRED CONTENT POINTS ARE ADDRESSED:**
- **CONSIDER THE COMPLETENESS OF COMMUNICATIVE PURPOSES IN THE COMMUNICATIVE ACHIEVEMENT SUBSCALE.**
- **FOCUS ON THE CRITERIA SPECIFIC TO EACH SUBSCALE. DO NOT PENALIZE ISSUES THAT BELONG TO OTHER SUBSCALES.**
- **ASSESS LANGUAGE ERRORS UNDER THE LANGUAGE SUBSCALE UNLESS THEY SIGNIFICANTLY IMPEDE COMMUNICATION IN OTHER SUBSCALES.**
- **DO NOT LET LANGUAGE ERRORS OR CLARITY ISSUES DUE TO LANGUAGE ERRORS AFFECT THE CONTENT SCORE.**
- **UNDER THE CONTENT SUBSCALE, IF THE STUDENT ADDRESSES ALL REQUIRED CONTENT POINTS, YOU MUST ASSIGN A SCORE OF 5 IN CONTENT, REGARDLESS OF ANY OTHER ISSUES, INCLUDING LANGUAGE ERRORS, IRRELEVANCES, OR MISINTERPRETATIONS DUE TO LANGUAGE ERRORS. DO NOT DEDUCT POINTS IN THE CONTENT SUBSCALE FOR ANY REASON OTHER THAN MISSING OR INADEQUATELY ADDRESSING REQUIRED CONTENT POINTS.**
- Only reply with the assessment criteria; don't add anything else.
- The feedback is given by you.

**Content Subscale:**

- **Band 5 (Score 5):**
  - **All content is relevant to the task.**
  - **All content points from the task are addressed appropriately.**
  - **The target reader is fully informed.**
  - **IMPORTANT: IF ALL REQUIRED CONTENT POINTS ARE MENTIONED, YOU MUST ASSIGN A CONTENT SCORE OF 5, REGARDLESS OF LANGUAGE ERRORS OR CLARITY ISSUES. DO NOT DEDUCT POINTS FOR ANY OTHER REASONS IN THE CONTENT SUBSCALE.**

- **Band 4 (Score 4):**
  - Performance shares features of Bands 3 and 5.

- **Band 3 (Score 3):**
  - **Minor irrelevances and/or omissions may be present.**
  - **Most content points are addressed.**
  - **The target reader is on the whole informed.**

- **Band 2 (Score 2):**
  - Performance shares features of Bands 1 and 3.

- **Band 1 (Score 1):**
  - **Irrelevances and misinterpretation of task may be present.**
  - **Some content points are addressed.**
  - **The target reader is minimally informed.**

**Communicative Achievement Subscale:**

- **Band 5 (Score 5):**
  - **Uses the conventions of the communicative task to hold the target reader’s attention and communicate straightforward ideas.**
  - **Maintains an appropriate tone and register for the task.**

- **Band 4 (Score 4):**
  - Performance shares features of Bands 3 and 5.

- **Band 3 (Score 3):**
  - **Uses the conventions of the communicative task in generally appropriate ways to communicate straightforward ideas.**
  - **Some awareness of tone and register, which is generally appropriate.**

- **Band 2 (Score 2):**
  - Performance shares features of Bands 1 and 3.

- **Band 1 (Score 1):**
  - **Produces text that communicates simple ideas in simple ways.**
  - **Limited or inconsistent awareness of tone and register.**

**Organisation Subscale:**

- **Band 5 (Score 5):**
  - **Text is generally well-organised and coherent.**
  - **Uses a variety of linking words and cohesive devices.**

- **Band 4 (Score 4):**
  - Performance shares features of Bands 3 and 5.

- **Band 3 (Score 3):**
  - **Text is connected and coherent.**
  - **Uses basic linking words and a limited number of cohesive devices.**

- **Band 2 (Score 2):**
  - Performance shares features of Bands 1 and 3.

- **Band 1 (Score 1):**
  - **Text is connected using basic, high-frequency linking words.**
  - **Limited use of cohesive devices; ideas may be disjointed.**

**Language Subscale:**

- **Band 5 (Score 5):**
  - **Uses a range of everyday vocabulary appropriately, with occasional inappropriate use of less common lexis.**
  - **Uses a range of simple and some complex grammatical forms with a good degree of control.**
  - **Errors do not impede communication.**

- **Band 4 (Score 4):**
  - Performance shares features of Bands 3 and 5.

- **Band 3 (Score 3):**
  - **Uses everyday vocabulary generally appropriately, while occasionally overusing certain lexis.**
  - **Uses simple grammatical forms with a good degree of control.**
  - **While errors are noticeable, meaning can still be determined.**

- **Band 2 (Score 2):**
  - Performance shares features of Bands 1 and 3.

- **Band 1 (Score 1):**
  - **Uses basic vocabulary reasonably appropriately.**
  - **Uses simple grammatical forms with some degree of control.**
  - **Errors may impede meaning at times.**

**Additional Guidance:**

- **Identify the content points required by the task.**
- **Ensure that each content point is addressed in the student's response.**
- **Use a checklist to confirm that all required content points are addressed.**
- **Do NOT penalise for language errors in the Content subscale.**
- **Assess the student's ability to communicate their ideas effectively in the Communicative Achievement subscale.**
- **Evaluate the coherence and cohesion of the text in the Organisation subscale.**
- **Assess the range and accuracy of vocabulary and grammar in the Language subscale.**

    **<EXAMPLE>**
    **<TaskAssessment>**
    **Rubric:**
You see this notice on an English-language website.

Articles wanted!

FILMS

What kind of films do you enjoy?

Do you prefer watching them at the cinema or at home? Why?

Write an article answering these questions and we will put it on our website!

Write your article.
    **</TaskAssessment>**
    **<TaskAnswer>**
    Hi, my name is Antem. I likes watch films, but there are little cinemas in our city.

I prefer watch fantastic films at the cinema in 3D, because they realise and interesting. My favourite filmes are ‘star wars’, ‘hobbit’ and, Harry Potter and the half-blood prince’.

I prefer watch films at the cinema park in Alatir. If you ask: Why do you like watch film at the cinema park? I answer, that I like eat pop-corn and ice-cream. Else, I like go to the cinema with my family. It’s very funny! We go to the cinema one time a month, and two times a week on holiday.

    **</TaskAnswer>**
    **<ExampleResult>**
    {{
        "Content": {{
            "score": 5,
            "feedback": "All content is relevant to the task. The target reader is fully informed about what sort of films the candidate enjoys watching, fantastic films, where they like watching them, at the cinema park, and why, I like eat pop-corn and ice-cream."
        }},
        "CommunicativeAchievement": {{
            "score": 2,
            "feedback": "The conventions of article writing, such as describing personal experiences and expressing opinions, are used generally appropriately to communicate simple ideas, I prefer watch; My favourite filmes are. There is an attempt to engage the reader directly through the use of phrases such as, If you ask … I answer, that …; It’s very funny!"
        }},
        "Organisation": {{
            "score": 2,
            "feedback": "The text is connected and coherent. Basic linking words are used within the text to connect some of the ideas, because; and. There is some attempt to paragraph and punctuate the text, but this is not always successful as there are single sentences which are not integrated into the text, I prefer watch filmes at the cinema park in Alatir."
        }},
        "Language": {{
            "score": 2,
            "feedback": "A range of everyday vocabulary, suitable for the topic, is used generally appropriately, fantastic films; cinema park; pop-corn and ice-cream; one time a month; two times a week on holiday. Simple grammatical forms, such as the present simple, are used with some degree of control, although there is not much variety in terms of tenses, I likes watch; I prefer watch. Errors are noticeable, but meaning can still be determined, they realise and interesting; Else, I like go"
        }}
    }}
    **</ExampleResult>**
    **</EXAMPLE>**

    **Task to Evaluate:**
    **<TaskAssessment>**
    {question}
    **</TaskAssessment>**
    **<TaskAnswer>**
    {answer}
    **</TaskAnswer>**

    **Extra Information:**
    - In the output, maintain camel case and spaces format for criteria in the output.
    - Do not take into consideration or transcribe struck-through words.
    - Candidates are asked to write approximately 100 words. Writing approximately the right number of words is an integral part of task achievement. If significantly fewer words are written, this is likely to mean that the task has not been successfully completed. If significantly more words are written, there is likely to be irrelevance, repetition of ideas, or poor organisation.
"""

In [12]:
prompt = [
            {
        "role": "system",
        "content": f"""
        You are a world-class B1 Preliminary Cambridge writing examiner with over 10 years of experience.
        As a certified Cambridge English examiner, you have in-depth knowledge of the CEFR standards, especially for the B1 level.
        You provide detailed, constructive feedback that highlights strengths and areas for improvement, using clear and supportive language. Your goal is to help learners progress by offering actionable advice while encouraging their efforts.
        You are asked to evaluate students' article writing in response to a task and provide feedback.
        """
             },
            {
                "role": "user",
                "content": get_user_prompt(question,student_answer)
               
            }
        ]

A Partir de Aca ejecutas SIEMPRE !!!!

In [13]:
print(prompt[0]['content'])
print(prompt[1]['content'])


        You are a world-class B1 Preliminary Cambridge writing examiner with over 10 years of experience.
        As a certified Cambridge English examiner, you have in-depth knowledge of the CEFR standards, especially for the B1 level.
        You provide detailed, constructive feedback that highlights strengths and areas for improvement, using clear and supportive language. Your goal is to help learners progress by offering actionable advice while encouraging their efforts.
        You are asked to evaluate students' email or letter writing in response to a task and provide feedback.
        

The task requires candidates to write an article of approximately 100 words in response to an announcement or prompt, typically from a magazine or website. The input outlines the topic of the article and usually includes a set of questions that candidates must address in their writing.

Candidates are expected to write an engaging and informative article that is suitable for the specified audi

In [14]:
service.temperature=0.7

In [15]:
ai_response, spent_tokens = service.parse(WritingResult,prompt=prompt)


for criterion_score in ai_response.criteria:
    print(f" {criterion_score.criteria}, Score: {criterion_score.score},   feedback: {criterion_score.feedback} \n")



#df = convert_openai_output_to_df(json.loads(ai_response))
#df

 Content, Score: 5,   feedback: All content is relevant to the task. The target reader is fully informed about what makes the writer laugh, such as their cat and jokes with friends and family, and why they think it's good to laugh a lot, as it eases stress and promotes happiness. 

 CommunicativeAchievement, Score: 4,   feedback: The article maintains an appropriate tone and engages the reader by sharing personal experiences and opinions about laughter. It successfully uses the conventions of article writing to communicate ideas in an engaging manner, though minor improvements could enhance engagement further. 

 Organisation, Score: 4,   feedback: The text is generally well-organised and coherent, with ideas logically connected. However, the use of linking words and cohesive devices could be expanded to improve the flow between sentences and paragraphs. 

 Language, Score: 4,   feedback: A range of everyday vocabulary is used appropriately, with effective use of simple and some comple